<a href="https://colab.research.google.com/github/khalidelboray/ML_Course/blob/master/Project/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip -q install requests pandas numpy matplotlib plotly scikit_learn seaborn kaggle wordcloud zipfile36 swifter requests-html tqdm

In [ ]:
kaggle_path = '../kaggle.json'
IN_COLAB =  'google.colab' in str(get_ipython())
IN_COLAB

In [ ]:
if IN_COLAB:
    print("Looks like you are running colab, an upload form will show up now to upload your kaggle.json file..")
    from google.colab import files 
    files.upload()
    kaggle_path = './kaggle.json'

In [ ]:
import os
from pathlib import Path
from os.path import abspath
HOME_PATH = Path.home()
if not os.path.isdir(f'{HOME_PATH}/.kaggle'):
    os.makedirs(f'{HOME_PATH}/.kaggle/')
if os.path.isfile(kaggle_path):
    from shutil import copyfile
    copyfile(kaggle_path, f'{HOME_PATH}/.kaggle/kaggle.json')
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
!kaggle datasets download -d stefanoleone992/imdb-extensive-dataset

In [ ]:
if not os.path.isdir('data'):
    os.makedirs('data')

In [ ]:
import sys
if sys.version_info >= (3, 6):
    import zipfile
else:
    import zipfile36 as zipfile
pz = open('imdb-extensive-dataset.zip', 'rb')
packz = zipfile.ZipFile(pz)
for name in packz.namelist():
    packz.extract(name, 'data')
pz.close()
os.remove('imdb-extensive-dataset.zip')

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
pd.options.display.max_columns = 999
print('max_rows:' , pd.options.display.max_rows)
print('max_columns:' , pd.options.display.max_columns)
%matplotlib inline

In [ ]:
movies = pd.read_csv('./data/IMDb movies.csv')

In [ ]:
movies.head()

In [ ]:
movie_urls = movies['imdb_title_id'].apply(lambda x: f"http://www.imdb.com/title/{x}").values
movie_urls

In [ ]:
from requests_html import AsyncHTMLSession
session = AsyncHTMLSession()
async def get_image_url(movie_url):
    resp = await session.get(movie_url)
    img_elem = resp.html.find('.ipc-image', first=True)
    link = img_elem.attrs['srcset'].split('w,')[-1].strip().split(' ')[0].strip()
    return link

In [ ]:
from tqdm import tqdm
import asyncio
import tqdm.asyncio
import urllib
movies['image'] =  pd.Series(dtype=str)

if not os.path.isdir('./images'):
    os.makedirs('./images')
    
async def process_url(url):
    mid = url.split('/')[-1]
    location = f"./images/{mid}.jpg"
    url = get_image_url(url)
    with open (location,'wb') as out:
        out.write(await session.get(url).content)
    movies.loc[movies['imdb_title_id'] == mid , 'image'] = location
    
async def process_all(urls):
    tasks = []
    for url in urls:
        task = asyncio.create_task(process_url(url))
        tasks.append(task)
        
    all_paths = [
        await process_url(url=url)
        async for url in tqdm.asyncio.tqdm(urls)
    ]


# for url in tqdm(movie_urls):
#     mid = url.split('/')[-1]
#     location = f"./images/{mid}.jpg"
#     url = get_image_url(url)
#     with open (location,'wb') as out:
#         out.write(session.get(url).content)
#     movies.loc[movies['imdb_title_id'] == mid , 'image'] = location

In [ ]:
!python3 -m pip install nest_asyncio
import nest_asyncio

nest_asyncio.apply()


In [ ]:
asyncio.run(
        process_all(urls=movie_urls[0:10]))

In [ ]:
movies['image'].head()